#   Mood Detection and movie Recommendation using K nearest Neighbour

### Importing Modules

In [2]:
!!pip install opencv-python
import cv2
import numpy as np
import os

In [3]:
d={}
d['joy']=0
d['anger']=0
d['surprise']=0
d['fear']=0
d['sad']=0

### function 2 check similarity of 2 images

In [4]:
def distance(v1, v2):
    return np.sqrt(((v1-v2)**2).sum())

### KNN function

In [5]:
def knn(train, test, k=5):
    dist = []
    for i in range(train.shape[0]):
        ix = train[i, :-1]
        iy = train[i, -1]
        d = distance(test, ix)
        dist.append([d, iy])
    dk = sorted(dist, key=lambda x: x[0])[:k]
    labels = np.array(dk)[:, -1]
    output = np.unique(labels, return_counts=True)
    index = np.argmax(output[1])
    return output[0][index]


In [6]:
cam = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('C:/Users/KARAN KAPOOR/AppData/Local/Programs/Python/Python37-32/Lib/site-packages/cv2/data/haarcascade_frontalface_alt.xml')
skip= 0
dataset_path ="E:/Facedata/"
class_id = 0 #label for given file
names = {}
face_data = []
labels = []

### Data Preparation

In [7]:
for fx in os.listdir(dataset_path):
    if fx.endswith(".npy"): #a numpy file
        names[class_id] = fx[:-4]
        print("Loading file ",fx)
        data_item = np.load(dataset_path+fx) #file name and .npy
        face_data.append(data_item)
        target = class_id*np.ones((data_item.shape[0],)) #matrix of ones
        class_id +=1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis=0)
face_label= np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_label.shape)


Loading file  anger.npy
Loading file  fear.npy
Loading file  joy.npy
Loading file  sad.npy
Loading file  surprise.npy
(202, 30000)
(202, 1)


In [ ]:
trainset = np.concatenate((face_dataset,face_label),axis=1)
print(trainset.shape)

In [ ]:
count=0
while True:
    ret,frame = cam.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)

    if(len(faces)==0): #if no face is seen, the camera hangs up rather than clossing with empty()! error
        cv2.imshow("Faces",frame)
        continue 

    for face in faces:
        x,y,w,h = face

        offset = 10
        face_selection = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_selection = cv2.resize(face_selection,(100,100))


        out = knn(trainset,face_selection.flatten())
        pred_name = names[int(out)]	
        d[pred_name]=d[pred_name]+1
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,125),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),5)

    count=count+1;
    cv2.imshow("Faces",frame)
    
    key=cv2.waitKey(1) & 0xFF

    if key==ord('q') or count>=100:
        break

In [10]:
s=""
max1=0
if(d['joy']>max1):
    max1=d['joy']
    s="joy"
if(d['anger']>max1):
    max1=d['anger']
    s="anger"
if(d['surprise']>max1):
    max1=d['surprise']
    s="surprise"
if(d['fear']>max1):
    max1=d['fear']
    s="fear"
if(d['sad']>max1):
    max1=d['sad']
    s="sad"
print(s)

fear


In [11]:
    if s=="joy":
        print('3 Idiots')
        print('Dhammal')
        print('Hera Pheri')
        print('Golmaal')
        print('Munna Bhai MBBS')
    if s=="anger":
        print('Wanted')
        print('Krrish')
        print('Kick')
        print('Dhoom 2')
        print('Agneepath')
    if s=="surprise":
        print('Yesterday')
        print('The loin King')
        print('Crawl')
        print('Stubber')
        print('Shaft')
    if s=="sad":
        print('Atonement')
        print('The Road')
        print('Remember me')
        print('The champ')
        print('The last Song')
    if s=="fear":
        print('A quite place')
        print('Halloween')
        print('It')
        print('Hush')
        print('Us')



A quite place
Halloween
It
Hush
Us


In [12]:
cam.release()
cv2.destroyAllWindows()